# DeepSeek GRPO 训练流程
在本文件中，我们将手动实现一个 GRPO 并使用它为 Qwen2.5 模型添加类似于 DeepSeek 的推理能力，旨在了解 GRPO 的技术原理并掌握其基本实现方法。

## 1 初始化模型并构建聊天模板
### 1.1 安装所需依赖

In [1]:
%pip install modelscope
%pip install torch
%pip install transformers
%pip install trl

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
  Using cached http://mirrors.aliyun.com/pypi/packages/61/e8/4e40eb57d77d64656067528c59d130120d3e3ea343fba0aaf50776111bb1/modelscope-1.23.2-py3-none-any.whl (5.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Using cached http://mirrors.aliyun.com/pypi/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762aeb1c1184ed1f6b14/tqdm-4.67.1-py3-none-any.whl (78 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.1/146.1 kB 1.4 MB/s eta 0:00:0000:01

[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: htt

### 1.2 下载模型
为了尽可能减少训练成本，我们使用参数量为 0.5B 的 Qwen2.5 模型作为基座模型。

In [2]:
import os
# 在本地创建文件夹用于保存模型参数
os.makedirs("./Qwen2.5-0.5B-Instruct", exist_ok=True)

In [3]:
# 从 modelscope 库下载 Qwen2.5 模型到指定文件夹
!modelscope download --model Qwen/Qwen2.5-0.5B-Instruct --local_dir ./Qwen2.5-0.5B-Instruct

^C


### 1.3 加载模型与分词器

In [3]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

# 指定模型所在目录
model_name = "./Qwen2.5-0.5B-Instruct"

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype="auto",
    device_map="auto",
)

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

/home/deepseek_tutorial/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


`AutoModelForCausalLM.from_pretrained()`:

**核心功能：**

- 从指定路径加载模型
- 支持动态选择计算精度并分配设备

**参数解析：**

- `pretrained_model_name_or_path`: 基座模型路径。
- `torch_dtype`: 选择张量数据类型。"auto"表示根据硬件能力自动选择张量数据类型(float32, float16, bfloat16)，在保持精度的同时最大化计算效率。
- `device_map`: 选择设备分配策略。"auto"表示根据硬件能力自动分配 CPU/GPU 资源。

`AutoTokenizer.from_pretrained()`:

**核心功能：**

- 从指定路径加载文件的 tokenizer(分词器)

### 1.4 创建对话消息
平时我们在使用大语言模型应用进行对话时，在聊天框中输入的内容就称为 prompt。输入的 prompt 在被传输到后台服务器时，会被组织成`{"role": "user", "content": <你输入的 prompt>}`这样的形式（注：这种数据类型叫做`json`，大家之后在数据处理任务中会经常使用到它），用于表示这是用户发送的消息。

通常而言，在聊天这一应用场景中，每条消息均以 json 格式定义，包含两个关键字段：

- `role`: 标识消息来源，支持三种标准角色：
  - `user`: 用户发送的消息（输入的 prompt）
  - `assistant`: 模型生成的回复
  - `system`: 系统级指令（用于预设模型行为，通常仅出现在首条消息）
- `content`: 消息的文本内容

因此，用户和系统之间的消息你来我往，便能够构成如下所示的消息历史：

```json
messages = [
  {"role": "system", "content": "......"},
  {"role": "user", "content": "......"},
  {"role": "assistant", "content": "......"},
  {"role": "user", "content": "......"},
  {"role": "assistant", "content": "......"},
  ......
]
```

每一次回答时，消息历史中的所有内容都会被作为模型输入，因此你会发现模型总能够“记住”你们之间说过的话（实际上，模型输入有一定的长度限制，即“上下文窗口长度”，通常按照 token 数量来计算，比如 4K/16K tokens，超出长度的部分会被自动截断）。

In [5]:
prompt = "Joy can read 8 pages of a book in 20 minutes. How many hours will it take her to read 120 pages?"
messages = [
    {"role": "user", "content": prompt}
]
print("\n===== 原始消息结构 =====\n", messages)


===== 原始消息结构 =====
 [{'role': 'user', 'content': 'Joy can read 8 pages of a book in 20 minutes. How many hours will it take her to read 120 pages?'}]


上面说到，模型和用户的对话历史会被组织成 json 列表的形式传输给模型，接下来便需要模型对上述消息作出回答。

我们知道，语言模型所做的任务本质上就是“文字接龙”，而聊天是一项比较特殊的“文字接龙”，因为它加入了身份的切换，从而让输出看起来像是一场对话。因此同其他任务一样，在聊天这项任务中，模型在作出回答时，首先也需要先做 tokenize 操作，也就是将大段的文字切分成一个又一个模型认识的小字段，也就是 token。

为了将`messages`格式化为模型输入，需要调用`tokenizer.apply_chat_template()`方法。并为该方法设定了两个参数值：`tokenize=False`和`add_generation_prompt=True`。

怎么理解这两个参数呢？

首先，`add_chat_template()`会将传入的 json 列表`messages`拼接为一段文本。接着，如果`tokenize=True`，则这一段文本会被直接被切分成 tokens，最终输出由各个 token 的 id 组成的数字序列；如果`tokenize=False`，则会保留文本原本的样式。具体区别可以对比下面代码块中的`text`和`text_1`。

然后是`add_generation_prompt`这个参数。简单来说，如果`add_generation_prompt=True`，那么每次在`<|im_start|>user......<|im_end|>`后，都会自动接一个`<|im_start|>assistant`，用来告诉模型：“嘿，用户已经说完了，现在该你回答了”；如果`add_generation_prompt=False`，则不会自动接这个提示词。不过一般来讲，在实操中我们都会选择`add_generation_prompt=True`，就像上面所说的，我们是在用一个“文字接龙”的机器模拟“聊天”，因此如果不刻意地给模型一些提示，它可能会接着补充用户的内容，而不是“发表”自己的意见。具体区别可以对比下面代码块中的`text`和`text_2`。

In [6]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

text_1 = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True
)

text_2 = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=False
)

print("\n===== text =====\n", text)
print("\n===== text_1 =====\n", text_1)
print("\n===== text_2 =====\n", text_2)


===== text =====
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Joy can read 8 pages of a book in 20 minutes. How many hours will it take her to read 120 pages?<|im_end|>
<|im_start|>assistant


===== text_1 =====
 [151644, 8948, 198, 2610, 525, 1207, 16948, 11, 3465, 553, 54364, 14817, 13, 1446, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 79771, 646, 1349, 220, 23, 6816, 315, 264, 2311, 304, 220, 17, 15, 4420, 13, 2585, 1657, 4115, 686, 432, 1896, 1059, 311, 1349, 220, 16, 17, 15, 6816, 30, 151645, 198, 151644, 77091, 198]

===== text_2 =====
 <|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Joy can read 8 pages of a book in 20 minutes. How many hours will it take her to read 120 pages?<|im_end|>



### 1.5 将文本格式化为模型输入
截至目前我们已经获得了经过`apply_chat_template`格式化后的`text`，接下来需要将文本格式化为模型能够理解的输入，这一过程需要使用的是`tokenizer`，最终返回的则是一个列表，列表由每个 token 在词汇表中对应的 id 组成，也就是代码块输出的`input_ids`。

此外，输出中还包含`attention_mask`和`device`，其中`attention_mask`用来标识哪些位置是有效 token（1=有效，0=填充位）；`device`将张量移动到模型所在设备，因为模型参数和张量必须在同一设备上才能计算。

In [7]:
model_inputs = tokenizer(
    [text],                # 输入文本（列表形式，支持批量输入）
    return_tensors="pt"    # 返回 PyTorch 张量
    ).to(model.device)     # 将张量迁移到模型所在设备
print(model_inputs)

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,   1207,  16948,     11,   3465,
            553,  54364,  14817,     13,   1446,    525,    264,  10950,  17847,
             13, 151645,    198, 151644,    872,    198,  79771,    646,   1349,
            220,     23,   6816,    315,    264,   2311,    304,    220,     17,
             15,   4420,     13,   2585,   1657,   4115,    686,    432,   1896,
           1059,    311,   1349,    220,     16,     17,     15,   6816,     30,
         151645,    198, 151644,  77091,    198]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


### 1.6 获取模型输出并转化为文本
接下来我们将输入喂给模型并获取模型输出，第一步先是获取模型新生成的 token 的 id 序列。

In [8]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512    # 限制模型生成新的 token 数不超过 512
)
# 遍历每个输入样本的 input_ids 和对应的生成结果 output_ids
# 对每个样本，计算输入长度 len(input_ids)，截取生成结果中从该位置到末尾的部分
generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids
    in zip(model_inputs.input_ids, generated_ids)
]
print(generated_ids)

[tensor([  1249,   8253,   1246,   1293,    432,    686,   1896,  27138,    311,
          1349,    220,     16,     17,     15,   6816,     11,    582,   1156,
          1184,    311,   1477,    700,   1059,   5290,   4379,    304,   6816,
           817,   9383,     13,   1205,   1414,    429,   1340,  15804,    220,
            23,   6816,    304,    220,     17,     15,   4420,     13,  15277,
            11,   1059,   5290,   4379,    374,   1447,     59,   9640,     59,
          1318,     90,  31899,   4379,     92,    284,   1124,  37018,     90,
            23,   1124,   1318,     90,   6816,   3417,     90,     17,     15,
          1124,   1318,     90,   4420,   3417,    284,   1124,  37018,     90,
            17,   1124,   1318,     90,   6816,   3417,     90,     20,   1124,
          1318,     90,   4420,  11248,     59,   2533,   1986,   3363,    429,
         27138,  15804,    518,    264,   4379,    315,   1124,  11520,  37018,
            90,     17,  15170,     20,

然后将 id 转化为对应的 token 文本，最终组合成一个字符串，即模型的回答。

`skip_special_tokens=True`表明跳过特殊标记如`<|im_start|>`,`<|im_end|>`等，这是因为特殊标记虽然对模型有意义，但对用户无意义，所以通常需要跳过。

In [9]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

To determine how long it will take Joy to read 120 pages, we first need to find out her reading rate in pages per minute. We know that she reads 8 pages in 20 minutes. Therefore, her reading rate is:

\[
\text{Reading rate} = \frac{8 \text{ pages}}{20 \text{ minutes}} = \frac{2 \text{ pages}}{5 \text{ minutes}}
\]

This means that Joy reads at a rate of \(\frac{2}{5}\) pages per minute. To find out how long it takes her to read 120 pages, we use the formula:

\[
\text{Time} = \frac{\text{Total pages}}{\text{Reading rate}}
\]

Substituting the known values:

\[
\text{Time} = \frac{120 \text{ pages}}{\frac{2}{5} \text{ pages/minute}} = 120 \times \frac{5}{2} = 60 \times 5 = 300 \text{ minutes}
\]

Since there are 60 minutes in an hour, we convert the time from minutes to hours:

\[
300 \text{ minutes} = \frac{300}{60} \text{ hours} = 5 \text{ hours}
\]

Therefore, it will take Joy \(\boxed{5}\) hours to read 120 pages.


至此，我们成功用原生的 Qwen2.5 大模型构建起了一个能够聊天的机器人，但是目前它的回答是糅合在一起的，并不具备像 DeepSeek 那样的推理能力，接下来我们就为它添加推理能力。
## 2 尝试用 prompt 注入方式为模型添加推理能力
说起为模型添加推理能力，我们首先能够想到的便是通过 prompt 提示模型先给出推理然后再给出答案，那么这种简单朴素的想法是否能够成功呢？让我们一探究竟。

这部分需要用到一个名为 gsm8k 的数据集，它是一个广泛用于评估语言模型数学推理能力的基准数据集，包含 8500 个高质量的小学数学应用题，每条数据均包含`question`和`answer`两项内容，且`answer`中不仅有问题的答案，还包含了问题的计算推导过程，因此该数据集常用来测试模型理解和处理数学推理能力。该数据集可以直接从`datasets`库中获取。



In [10]:
# autodl 添加学术加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [11]:
import datasets as hf_datasets
data = hf_datasets.load_dataset('openai/gsm8k', 'main')
print("\n===== gsm8k 数据信息 =====")
print(data,"\n")
print("\n===== gsm8k 训练集数据展示（第一条） =====")
print(data['train'][0])


===== gsm8k 数据信息 =====
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
}) 


===== gsm8k 训练集数据展示（第一条） =====
{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}


In [12]:
# 关闭学术加速，否则会对正常网络造成影响
!unset http_proxy && unset https_proxy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


仔细观察可以看到，`answer`被`####`分成了两部分，前半部分是`reasoning`，后半部分则是直接给出`answer`。现在你可能已经想象到我们最终要赋予给 Qwen2.5 的是一个怎样的推理能力了。

对！就是要让它在`reasoning`时给出细致的数学推理过程，在回答时直接给出`answer`。

![image-20250305195837974](https://asdfdasgasd.oss-cn-chengdu.aliyuncs.com/typora_pictures/20250305195838053.png)

接下来对原始的 gsm8k 数据集进行重构，剔除`answer`中的`reasoning`部分，修改后的结构如下所示：

```json
{
  "question": "对应原数据集中的 question",
  "answer": "仅包含原 answer 中 #### 之后的部分",
  "prompt":
  	[
  		{
  			"role": "system",
  			"content": "SYSTEM_PROMPT，旨在提示模型按照先给出 reasoning 再给出 answer 的格式回答",
			},
			{
        "role": "user",
        "content": "原数据集中的 question"
      }
  	]
}
```



In [13]:
from datasets import load_dataset, Dataset

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

def extract_hash_answer(text: str) -> str | None:
    """  
    从原始文本中提取 answer(#### 之后的部分)
    """
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split]
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    })
    return data

In [14]:
dataset = get_gsm8k_questions()
print("\n===== 重构后的数据集信息 =====")
print(dataset)
print("\n===== 重构后的第一条数据 =====")
print(dataset[0])


===== 重构后的数据集信息 =====
Dataset({
    features: ['question', 'answer', 'prompt'],
    num_rows: 7473
})

===== 重构后的第一条数据 =====
{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': '72', 'prompt': [{'content': '\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n', 'role': 'system'}, {'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'role': 'user'}]}


细心的你可能已经发现了，重构之后的数据集中，键`prompt`对应的值本身就是一个对话历史，因此我们可以直接把`prompt`取出来作为`messages`喂给大模型，并获取大模型的回答。而且这次，咱们的`prompt`中包含了`SYSTEM_PROMPT`，提示模型先给出推理再给出答案。

In [15]:
messages = dataset[0]['prompt']
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

To find out how many clips Natalia sold in total in April and May, we need to follow these steps:

1. Determine the number of clips sold in May.
2. Add the number of clips sold in April and May.

First, we know that Natalia sold 48 clips in April. In May, she sold half as many clips as she did in April. So, we calculate the number of clips sold in May as follows:

\[ \text{Clips sold in May} = \frac{\text{Clips sold in April}}{2} = \frac{48}{2} = 24 \]

Next, we add the number of clips sold in April and May to find the total:

\[ \text{Total clips sold} = \text{Clips sold in April} + \text{Clips sold in May} = 48 + 24 = 72 \]

So, Natalia sold a total of 72 clips in April and May. The final answer is:

\[
\boxed{72}
\]


然而结果并不理想，模型并没有诞生思考过程，且结果也没有按照我们限定的格式返回，看来只靠 prompt 这样的“口头要求”来约束模型回答还是远远不够的。

专业的事交给专业的“人”，GRPO 这项技术就是专门用来训练模型逻辑推理能力的一种强化学习框架，这项技术由 DeepSeek 团队于 2024 年 4 月在 DeepSeekMath 这篇文章中首次提出，由于本篇教学仅做简单科普，因此关于其数学原理我们暂不讨论，有兴趣的读者可参考 [原论文](https://arxiv.org/abs/2402.03300) 并深入学习。

## 3 GRPO 奖励函数组复现

先前我们已经将 gsm8k 数据集进行了重组，重组之后每条`prompt`都可以作为对话历史喂给模型从而获得输出。GRPO 做的事情是，对每一条回答都会进行评估，在当前的数学计算场景下，有两项评估指标，分别是“准确性评估”和“格式评估”。如果回答结果准确，则给予模型“准确性奖励”，否则不给予奖励；如果回答格式准确，则给予模型“格式奖励”，否则不给予奖励。

下面将一一实现需要的奖励函数，大都是硬性的文本匹配规则，可讲的不多，主要是解释代码的实现方式。

In [16]:
# 封装模型回答
completion=  [{'content': response}]
completions = [completion]
print(completion)

[{'content': 'To find out how many clips Natalia sold in total in April and May, we need to follow these steps:\n\n1. Determine the number of clips sold in May.\n2. Add the number of clips sold in April and May.\n\nFirst, we know that Natalia sold 48 clips in April. In May, she sold half as many clips as she did in April. So, we calculate the number of clips sold in May as follows:\n\n\\[ \\text{Clips sold in May} = \\frac{\\text{Clips sold in April}}{2} = \\frac{48}{2} = 24 \\]\n\nNext, we add the number of clips sold in April and May to find the total:\n\n\\[ \\text{Total clips sold} = \\text{Clips sold in April} + \\text{Clips sold in May} = 48 + 24 = 72 \\]\n\nSo, Natalia sold a total of 72 clips in April and May. The final answer is:\n\n\\[\n\\boxed{72}\n\\]'}]


In [17]:
def extract_xml_answer(text: str) -> str:
    """  
    从模型回答的文本中提取数学结果（<answer></answer>之间的内容）。
    """
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

In [22]:
# 正确性检验
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """  
    检查模型输出是否与正确答案相匹配，并根据匹配情况返回奖励分数。

    输入：
    prompts: 由数据集中各条数据的`prompt`组成的列表，即输入给模型的对话历史。
    completions: 由模型回答组成的列表。
    answer: 由数据集中各条数据的`answer`组成的列表，即问题的正确答案。

    输出：
    奖励分数列表。
    """
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print("========================================================")
    print('-'*20, " Question ", '-'*20, "\n", q)
    print('-'*20, " Answer ", '-'*20, "\n", answer)
    print('-'*20, " Response ", '-'*20, "\n", responses[0])
    print('-'*20, " Extracted ", '-'*20, "\n", extracted_responses[0])
    print("========================================================")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

实现：

- `responses = [completion[0]['content'] for completion in completions]`: 提取每个 competion 中的内容（即模型的输出）
- `q = prompts[0][-1]['content']`: 提取输入问题（prompt）的内容
- `extracted_responses = [extract_xml_answer(r) for r in responses]`: 用`extract_xml_answer()`方法从模型输出中提取出答案部分
- `print(...)`打印问题、正确答案、模型输出和提取的答案
- `return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]`: 比较提取出来的每个答案与正确答案。如果两者相同，则返回奖励 2.0；否则返回 0.0。奖励是一个列表，长度与模型的输出数目相同。

In [23]:
correctness_reward_func(prompts=[dataset[0]['prompt']],
                        completions=completions,
                        answer=dataset[0]['answer'])

--------------------  Question  -------------------- 
 Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
--------------------  Answer  -------------------- 
 72
--------------------  Response  -------------------- 
 To find out how many clips Natalia sold in total in April and May, we need to follow these steps:

1. Determine the number of clips sold in May.
2. Add the number of clips sold in April and May.

First, we know that Natalia sold 48 clips in April. In May, she sold half as many clips as she did in April. So, we calculate the number of clips sold in May as follows:

\[ \text{Clips sold in May} = \frac{\text{Clips sold in April}}{2} = \frac{48}{2} = 24 \]

Next, we add the number of clips sold in April and May to find the total:

\[ \text{Total clips sold} = \text{Clips sold in April} + \text{Clips sold in May} = 48 + 24 = 72 \]

So, Natalia sold a total of 72 clips in Apr

[0.0]

In [24]:
def int_reward_func(completions, **kwargs) -> list[float]:
    """  
    检查模型输出是否为有效整数，并根据结果给予奖励。

    输入：
    completions: 由模型回答组成的列表。

    输出：
    奖励分数列表。
    """
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

- `responses = [completion[0]['content'] for completion in completions]`: 提取模型回答的内容
- `extracted_responses = [extract_xml_answer(r) for r in responses]`: 从模型回答中提取答案
- `return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]`: 检查回答的答案是否为整数形式。如果是则给予奖励 0.5；否则不给予奖励。

In [25]:
int_reward_func(completions)

[0.0]

In [26]:
import re

# 强格式检验
def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """  
    检查模型输出是否符合严格的格式要求。

    输入：
    completions: 由模型回答组成的列表。

    输出：
    奖励分数列表。
    """
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

- ` pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"`: 正则表达式，确保输出格式严格符合要求
- `responses = [completion[0]["content"] for completion in completions]`: 从模型回答中提取问题答案
- `matches = [re.match(pattern, r) for r in responses]`: 通过正则表达式检查每个回答是否符合格式
- `return [0.5 if match else 0.0 for match in matches]`: 如果格式匹配，则给予奖励 0.5；否则不给予奖励

In [27]:
strict_format_reward_func(completions)

[0.0]

In [28]:
# 弱格式检验
def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """  
    检查模型的输出是否符合稍微宽松的格式要求。

    输入：
    completions: 由模型回答组成的列表。

    输出：
    奖励分数列表。
    """
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

- `pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"`: 定义了一个稍微宽松的表达式
- `responses = [completion[0]["content"] for completion in completions]`: 从模型回答中提取问题答案
- `matches = [re.match(pattern, r) for r in responses]`: 检查每个回答是否符合此格式
- `return [0.5 if match else 0.0 for match in matches]`: 如果格式匹配则给予奖励 0.5；否则不给予奖励

In [29]:
soft_format_reward_func(completions)

[0.0]

In [30]:
# 标签计数检验
def count_xml(text) -> float:
    """  
    计算文本中标签的出现次数，本根据他们的位置和频率分配奖励。

    输入：
    text: 计数文本。

    输出：
    count: 标签总分。
    """
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

- `count = 0.0`: 初始化计数器
- `if text.count("<reasoning>\n") == 1:`: 检查是否有且仅有一个`<reasoning>\n`标签
- `count += 0.125`: 条件满足则奖励 0.125
- `if text.count("\n</reasoning>\n") == 1:`: 检查是否有且仅有一个`\n<reasoning>`标签
- `count += 0.125`: 条件满足则奖励 0.125
- `if text.count("\n<answer>\n") == 1:`: 检查是否有且仅有一个`\n<answer>\n`标签
- `count += 0.125`: 条件满足则奖励 0.125
- `count -= len(text.split("\n</answer>\n")[-1])*0.001`: 如果在标签`\n<answer>\n`之后存在多余的文本，按照文本长度扣除一些奖励
- `if text.count("\n</answer>") == 1:`: 检查是否有且仅有一个`\n</answer>`标签
- `count += 0.125`: 条件满足则奖励 0.125
- `count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001`: 如果存在多余文本则根据其长度扣除一些奖励

In [31]:
count_xml(response)

0.0

In [32]:
# 结构符合度检验
def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    """  
    计算每个模型输出的 XML 结构符合度，并给予奖励。

    输入：
    completions: 由模型回答组成的列表。

    输出：
    奖励分数列表。
    """
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

- `contents = [completion[0]["content"] for completion in completions]`: 从模型回答中提取答案
- `return [count_xml(c) for c in contents]`: 把`count_xml()`输出的结果作为奖励分数并返回

In [33]:
xmlcount_reward_func(completions)

[0.0]

## 4 GRPO 训练流程

### 4.1 训练参数设定

In [34]:
# 从指定位置读取模型
model_name = "./Qwen2.5-0.5B-Instruct"
# 指定模型训练中间值的输出地址
output_dir="outputs/Qwen-0.5B-GRPO"
# 指定项目运行的名字
run_name="Qwen-0.5B-GRPO-gsm8k"

In [35]:
from trl import GRPOConfig

training_args = GRPOConfig(
    output_dir=output_dir,
    run_name=run_name,
    learning_rate=5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    logging_steps=1,
    bf16=True,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    num_generations=16,
    max_prompt_length=256,
    max_completion_length=200,
    num_train_epochs=1,
    save_steps=100,
    max_grad_norm=0.1,
    log_on_each_node=False,
    use_vllm=False,
    vllm_gpu_memory_utilization=.3,
    vllm_device="cuda:0",
    report_to="wandb"
)

### 4.2 加载模型和分词器

In [36]:
import torch

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=None
).to("cuda")

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

### 4.3 设置网页端监控

In [37]:
%pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: http://mirrors.aliyun.com/pypi/simple

[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [38]:
import wandb
wandb.login(key="b45a51a2aa3d2984d2ac2089d28e9c6c94538a3a")
wandb.init(project="deepseek_tutorial")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zhangbihan999 (zhangbihan999-sichuan-normal-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


### 4.4 开始模型训练

In [38]:
from trl import GRPOTrainer

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func],
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

# 每隔指定步将模型权重保存到指定文件夹
trainer.save_model(output_dir)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f79b8ceeb00>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f78bc803100, raw_cell="from trl import GRPOTrainer

trainer = GRPOTrainer.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bconnect.cqa1.seetacloud.com/home/Deepseek_tutorial/rewrite.ipynb#Y113sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f79b8ceeb00>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f78bc802620, execution_count=38 error_before_exec=None error_in_exec=[Errno 32] Broken pipe info=<ExecutionInfo object at 7f78bc803100, raw_cell="from trl import GRPOTrainer

trainer = GRPOTrainer.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bconnect.cqa1.seetacloud.com/home/Deepseek_tutorial/rewrite.ipynb#Y113sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

### 4.5 运行模型并查看结果

In [ ]:
grpo_model_name = "./outputs/Qwen-0.5B-GRPO"
model = AutoModelForCausalLM.from_pretrained(
    grpo_model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Joy can read 8 pages of a book in 20 minutes. How many hours will it take her to read 120 pages?"
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": prompt}
]

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]
print(response)

能够看出，对比此前原始模型的状态，现在的模型已经能够顺利产生思考过程，且严格按照规定格式回答问题。